In [1]:
import datasets
train_set = datasets.load_from_disk(dataset_path="./train")
print(train_set)
test_set = datasets.load_from_disk(dataset_path="./valid")
print(test_set[0])

word_train = []
y_train = []
for i in range(len(train_set)):
    word_train.append(train_set[i]["content"])
    y_train.append(train_set[i]["personality"])

word_test = []
y_test = []
for i in range(len(test_set)):
    word_test.append(test_set[i]["content"])
    y_test.append(test_set[i]["personality"])

import re
import copy
import nltk
print(word_train[0])
s = re.split('[|,.":!@ ]', word_train[0])
l = []
for i in s:
    if i != '':
        l.append(i)
for j in range(len(l)):
    l[j] = l[j].lower()
print(l)

word_train_split = copy.deepcopy(word_train)
word_test_split = copy.deepcopy(word_test)
def sentence_split(words):
    for k in range(len(words)):
        s = re.split('[|,.":!@ ]', words[k])
        l = []
        for i in s:
            if i != '':
                l.append(i)
        for j in range(len(l)):
            l[j] = l[j].lower()
        words[k] = l


sentence_split(word_test_split)
sentence_split(word_train_split)


Dataset({
    features: ['content', 'personality'],
    num_rows: 58972
})
{'content': "OH!The shame!It's unbearable.:P|||Aha...Well if it weren't for my superiority complex,I'd probably be down on the ground,all neurotic and depressive,locked up in an asylum.You need it in order to develop past your initial sphere of...|||Well excuse me for being such a drag.If it truly were a superiority complex,then I wouldn't be curious about the opinion of other people when it comes to understanding them.I just stated above that...|||I consider myself to be rather complicated.The mbti,jung's studies,psychology in general and philosophy inspired me to reach a certain conclusion related to people:if some are dumb,then it's not...|||I sometimes do simulate a large variety of things in my head resulting in intense feelings close to what I'd experience in reality.It's a bit complicated for me because while some people believe...|||I can't say I hate people.I always had trouble hating because I usually 

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
word = word_train_split[0]
documents = []
c = 0
for word in word_train_split:
    documents.append(TaggedDocument(words=word, tags=[c]))
    c += 1
model = Doc2Vec(documents, vector_size=100, window=5, min_count=3, workers=4)
model.train(documents, total_examples=model.corpus_count, epochs=20)
print(model.infer_vector(word_train_split[0]))

[-1.04709554e+00  1.46341324e+00 -1.10809493e+00 -8.62112492e-02
  6.06594026e-01  4.90204722e-01  5.44232689e-02 -3.62945408e-01
  7.15103075e-02  7.40818799e-01 -5.02325118e-01 -9.27270174e-01
 -3.42354804e-01  5.71782470e-01 -7.04888701e-02 -9.76805091e-01
  1.05928016e+00  1.23611100e-01 -7.65622854e-01 -1.16724384e+00
 -1.08449897e-02 -6.76296175e-01 -2.88110524e-01  1.38453841e+00
 -1.41313696e+00  1.31390452e-01 -9.74839851e-02 -4.90896046e-01
 -1.30485260e+00 -2.36398205e-01 -1.03306934e-01 -3.00992906e-01
  5.01909666e-02  9.09668729e-02  7.19120800e-01  2.15009972e-01
 -1.37476337e+00  6.18132055e-01 -1.88994870e-01 -4.38300341e-01
  5.64552844e-01  1.52786568e-01 -6.25036359e-01 -2.45671958e-01
  8.14079404e-01 -1.82049048e+00  2.79835075e-01  4.28210109e-01
  4.08728063e-01 -3.21805149e-01 -3.92532498e-01 -2.22940877e-01
 -7.86253452e-01  2.37745070e+00 -1.04262829e+00  1.51021192e-02
  3.67104709e-02 -2.05855027e-01 -3.29827219e-01 -1.06128109e+00
  6.80127501e-01  1.00467

In [3]:
for i in range(len(y_train)):
    l = [0, 0, 0, 0]
    s = str(y_train[i])
    if s[0] == 'I':
        l[0] = 1
    if s[1] == 'S':
        l[1] = 1
    if s[2] == 'T':
        l[2] = 1
    if s[3] == 'P':
        l[3] = 1
    y_train[i] = l
y1_train = []
y2_train = []
y3_train = []
y4_train = []
for i in range(len(y_train)):
    y1_train.append(y_train[i][0])
    y2_train.append(y_train[i][1])
    y3_train.append(y_train[i][2])
    y4_train.append(y_train[i][3])
for i in range(len(y_test)):
    l = [0, 0, 0, 0]
    s = str(y_test[i])
    if s[0] == 'I':
        l[0] = 1
    if s[1] == 'S':
        l[1] = 1
    if s[2] == 'T':
        l[2] = 1
    if s[3] == 'P':
        l[3] = 1
    y_test[i] = l
y1_test = []
y2_test = []
y3_test = []
y4_test = []
for i in range(len(y_test)):
    y1_test.append(y_test[i][0])
    y2_test.append(y_test[i][1])
    y3_test.append(y_test[i][2])
    y4_test.append(y_test[i][3])

    
x_train = []
x_test = []
for i in range(len(word_train_split)):
    x_train.append(model.infer_vector(word_train_split[i]))
for i in range(len(word_test_split)):
    x_test.append(model.infer_vector(word_test_split[i]))

In [10]:

import numpy as np
x_train = np.array(x_train)
x_test = np.array(x_test)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
#nlp_model1 = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
nlp_model1 = LogisticRegression(C = 1.0, penalty = 'l2')
#nlp_model1 = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(16,), random_state=1, max_iter=10000)
nlp_model1.fit(x_train, y1_train)
y1_pred = nlp_model1.predict(x_test)
nlp_model1.fit(x_train, y2_train)
y2_pred = nlp_model1.predict(x_test)
nlp_model1.fit(x_train, y3_train)
y3_pred = nlp_model1.predict(x_test)
nlp_model1.fit(x_train, y4_train)
y4_pred = nlp_model1.predict(x_test)
y_test = np.array(y_test)
y_pred = np.zeros(y_test.shape)
for i in range(len(y_test)):
    y_pred[i][0] = y1_pred[i]
    y_pred[i][1] = y2_pred[i]
    y_pred[i][2] = y3_pred[i]
    y_pred[i][3] = y4_pred[i]
acc = 0
for i in range(len(y_test)):
    if y_test[i][0] == y_pred[i][0] and y_test[i][1] == y_pred[i][1] and y_test[i][2] == y_pred[i][2] and y_test[i][3] == y_pred[i][3]:
        acc += 1
acc = acc/(len(y_test))

In [11]:
print(acc)

0.23163659793814434
